# 00 - Ingestão de Dados Brutos (Bronze Layer)

Este notebook lê os arquivos JSON brutos do Yelp Dataset e os salva em formato Parquet na camada Bronze do Data Lake.

**Arquitetura Data Lake:**
- **Bronze**: Dados brutos em Parquet (cópia fiel dos JSONs)
- **Silver**: Dados limpos e transformados
- **Gold**: Datasets curados para modelagem

In [1]:
import os
from pyspark.sql import SparkSession
from datetime import datetime

In [2]:
# Criar sessão Spark com configuração de memória otimizada
spark = SparkSession.builder \
    .appName("Bronze Layer Ingestion") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

print(f"✅ Spark version: {spark.version}")

✅ Spark version: 3.5.0


In [3]:
# Configuração de caminhos
BASE_PATH = '/home/jovyan/work'
DATA_PATH = f'{BASE_PATH}/data'
BRONZE_PATH = f'{DATA_PATH}/bronze'

# Datasets a processar
DATASETS = {
    'business': 'yelp_academic_dataset_business.json',
    'review': 'yelp_academic_dataset_review.json',
    'user': 'yelp_academic_dataset_user.json',
    'tip': 'yelp_academic_dataset_tip.json',
    'checkin': 'yelp_academic_dataset_checkin.json'
}

print(f"📁 Diretório de dados: {DATA_PATH}")
print(f"🥉 Camada Bronze: {BRONZE_PATH}")
print(f"📊 Datasets a carregar: {len(DATASETS)}")

📁 Diretório de dados: /home/jovyan/work/data
🥉 Camada Bronze: /home/jovyan/work/data/bronze
📊 Datasets a carregar: 5


In [4]:
def ingest_to_bronze(dataset_name, filename):
    """
    Carrega um dataset JSON e salva em Parquet na camada Bronze.
    """
    print(f"\n{'='*70}")
    print(f"📥 Carregando: {dataset_name}")
    print(f"{'='*70}")
    
    start_time = datetime.now()
    
    # Ler JSON
    file_path = f"{DATA_PATH}/{filename}"
    print(f"📂 Lendo arquivo: {file_path}")
    
    # Otimização: Sampling para inferência de schema se necessário, mas read.json padrão é robusto
    df = spark.read.json(file_path)
    
    # Evitar count() antes de salvar para economizar memória/tempo
    # count = df.count()
    # print(f"📊 Total de registros: {count:,}")
    
    print(f"📋 Colunas: {len(df.columns)}")
    
    # Salvar em Parquet (Bronze)
    output_path = f"{BRONZE_PATH}/{dataset_name}"
    print(f"💾 Salvando em: {output_path}")
    
    df.write \
        .mode("overwrite") \
        .parquet(output_path)
    
    elapsed = (datetime.now() - start_time).total_seconds()
    print(f"✅ Concluído em {elapsed:.2f}s")
    
    return 0 # Retornando 0 pois removemos o count() custoso

In [5]:
print("\n🚀 INICIANDO INGESTÃO - CAMADA BRONZE\n")

results = {}

for dataset_name, filename in DATASETS.items():
    try:
        count = ingest_to_bronze(dataset_name, filename)
        results[dataset_name] = {'status': 'SUCCESS'}
    except Exception as e:
        print(f"❌ Erro ao carregar {dataset_name}: {str(e)}")
        import traceback
        traceback.print_exc()
        results[dataset_name] = {'status': 'FAILED', 'error': str(e)}

print(f"\n{'='*70}")
print("📊 RESUMO DA INGESTÃO - BRONZE LAYER")
print(f"{'='*70}")
for dataset, result in results.items():
    if result['status'] == 'SUCCESS':
        print(f"✅ {dataset:15s}: SUCESSO")
    else:
        print(f"❌ {dataset:15s}: FALHOU")


🚀 INICIANDO INGESTÃO - CAMADA BRONZE


📥 Carregando: business
📂 Lendo arquivo: /home/jovyan/work/data/yelp_academic_dataset_business.json
📋 Colunas: 14
💾 Salvando em: /home/jovyan/work/data/bronze/business
✅ Concluído em 9.65s

📥 Carregando: review
📂 Lendo arquivo: /home/jovyan/work/data/yelp_academic_dataset_review.json
📋 Colunas: 9
💾 Salvando em: /home/jovyan/work/data/bronze/review
✅ Concluído em 165.94s

📥 Carregando: user
📂 Lendo arquivo: /home/jovyan/work/data/yelp_academic_dataset_user.json
📋 Colunas: 22
💾 Salvando em: /home/jovyan/work/data/bronze/user
✅ Concluído em 151.85s

📥 Carregando: tip
📂 Lendo arquivo: /home/jovyan/work/data/yelp_academic_dataset_tip.json
📋 Colunas: 5
💾 Salvando em: /home/jovyan/work/data/bronze/tip
✅ Concluído em 5.32s

📥 Carregando: checkin
📂 Lendo arquivo: /home/jovyan/work/data/yelp_academic_dataset_checkin.json
📋 Colunas: 2
💾 Salvando em: /home/jovyan/work/data/bronze/checkin
✅ Concluído em 6.67s

📊 RESUMO DA INGESTÃO - BRONZE LAYER
✅ business    

In [6]:
print("\n🔍 VERIFICAÇÃO DA CAMADA BRONZE\n")

for dataset_name in DATASETS.keys():
    try:
        path = f"{BRONZE_PATH}/{dataset_name}"
        df = spark.read.parquet(path)
        count = df.count()
        print(f"📦 {dataset_name:20s}: {count:>10,} registros")
        print(f"   Colunas: {df.columns[:5]}..." if len(df.columns) > 5 else f"   Colunas: {df.columns}")
        print()
    except Exception as e:
        print(f"❌ Erro ao verificar {dataset_name}: {str(e)}\n")


🔍 VERIFICAÇÃO DA CAMADA BRONZE

📦 business            :    150,346 registros
   Colunas: ['address', 'attributes', 'business_id', 'categories', 'city']...

📦 review              :  6,990,280 registros
   Colunas: ['business_id', 'cool', 'date', 'funny', 'review_id']...

📦 user                :  1,987,897 registros
   Colunas: ['average_stars', 'compliment_cool', 'compliment_cute', 'compliment_funny', 'compliment_hot']...

📦 tip                 :    908,915 registros
   Colunas: ['business_id', 'compliment_count', 'date', 'text', 'user_id']

📦 checkin             :    131,930 registros
   Colunas: ['business_id', 'date']



In [7]:
spark.stop()
print("\n✅ INGESTÃO BRONZE CONCLUÍDA COM SUCESSO!")
print("\n💡 Próximos passos:")
print("   1. Execute o notebook 01_business_analysis.ipynb (Silver Layer)")
print("   2. Os dados brutos estão em formato Parquet na camada Bronze")
print("   3. Todas as transformações serão feitas com Spark")


✅ INGESTÃO BRONZE CONCLUÍDA COM SUCESSO!

💡 Próximos passos:
   1. Execute o notebook 01_business_analysis.ipynb (Silver Layer)
   2. Os dados brutos estão em formato Parquet na camada Bronze
   3. Todas as transformações serão feitas com Spark
